In [ ]:
import os
import re
import sys
import glob
import pickle
from time import strftime, localtime

import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
plt.rc('font', **{'family': 'sans-serif', 'size': 9})

if '..' not in sys.path:
    sys.path.append('..')
from deep_utils import *

%matplotlib inline

In [ ]:
def plot_data(data_folder, H_values, var_names,
              gen_id=None, area_id=None, gen_areas_map=None,
              gen_Pnom=None, bus_id=None, add_omega_ref=True,
              xlim_distr = None, ylim=[-4,4]):
    if gen_id is None and (bus_id is None or area_id is None):
        raise Exception('One of gen_id or (bus_id,area_id) must be not None')
    if gen_id is not None and (bus_id is not None or area_id is not None):
        raise Exception('One of gen_id or (bus_id,area_id) must be None')

    H_values = np.array(H_values)
    
    if gen_id is not None:
        inertia = {'training': H_values}
        time, x, y = load_data_generators([data_folder], [gen_id], inertia, var_names,
                                          max_block_size=500, use_tf=False)
        idx_min, _ = np.where(y['training'] == H_values.min())
        idx_max, _ = np.where(y['training'] == H_values.max())
        inertia_units = 's'
    else:
        area_inertia = 'energy'
        inertia_units = 'GW s'
        data_files = []
        for data_file in glob.glob(data_folder + '/*_training_set.h5'):
            _, _, areas_H = read_area_values(data_file, gen_areas_map, gen_Pnom, area_inertia)
            H = areas_H[area_id]
            if H in H_values:
                data_files.append(data_file)
        time, x, y = load_data_areas({'training': data_files}, var_names,
                                     gen_areas_map, gen_Pnom,
                                     area_inertia, max_block_size=1000,
                                     add_omega_ref = add_omega_ref, use_tf=False)
        idx_min, _ = np.where(y['training'] == H_values.min())
        idx_max, _ = np.where(y['training'] == H_values.max())

    x_train_mean = np.mean(x['training'], axis=(1,2))
    x_train_std = np.std(x['training'], axis=(1,2))
    for i,(m,s) in enumerate(zip(x_train_mean, x_train_std)):
        x['training'][i,:,:] = (x['training'][i,:,:] - m) / s
    
    N_vars = len(var_names)
    fig = plt.figure(figsize=(11,2 * N_vars))
    gs = fig.add_gridspec(N_vars, 4)
    ax = []
    for i in range(N_vars):
        ax.append(fig.add_subplot(gs[i,:2]))
    for j in (2,3):
        for i in range(N_vars):
            ax.append(fig.add_subplot(gs[i,j]))

    if xlim_distr is not None:
        lim = xlim_distr
    else:
        lim = 0
    
    # variables used in FFT computation
    block_size = 5
    N_samples = block_size * x['training'].shape[-1]
    dt = np.diff(time[:2])[0]
    freq = fftfreq(N_samples, dt)[:N_samples//2]
        
    cmap = [[0.2,0.8,0.2], [0.8,0.2,0.8]] 
    for i in range(N_vars):
        ### actual traces
        ax[i].plot(time, x['training'][i, idx_min[:5], :].T, color=cmap[0], lw=1)
        ax[i].plot(time, x['training'][i, idx_max[:5], :].T, color=cmap[1], lw=1)

        ### distributions
        n,edges = np.histogram(np.ndarray.flatten(x['training'][i,:,:][idx_min,:]), \
                               bins=100, range=ylim, density=True)
        lim = n.max() if n.max() > lim else lim
        ax[i+N_vars].plot(n, edges[:-1], color=cmap[0], linewidth=1.2)
        n,edges = np.histogram(np.ndarray.flatten(x['training'][i,:,:][idx_max,:]), \
                               bins=100, range=ylim, density=True)
        lim = n.max() if n.max() > lim else lim
        ax[i+N_vars].plot(n, edges[:-1], color=cmap[1], linewidth=1.2)
        ax[i+N_vars].set_yticklabels([])
        
        ### FFTs
        N_traces = idx_min.size // block_size
        Xf = np.zeros((N_traces, N_samples), dtype=np.complex64)
        for j in range(N_traces):
            start_trace = j * block_size
            X = np.vstack(x['training'][i, idx_min[start_trace : start_trace + block_size], :]).flatten()
            Xf[j,:] = fft(X)
        Xf = 2.0 / N_samples * np.abs(Xf[:, 0 : N_samples // 2])
        ax[i+2*N_vars].semilogy(freq, Xf.mean(axis=0), color=cmap[0], lw=1, label=f'H={H_values[0]:g} {inertia_units}')
        N_traces = idx_max.size // block_size
        Xf = np.zeros((N_traces, N_samples), dtype=np.complex64)
        for j in range(N_traces):
            start_trace = j * block_size
            X = np.vstack(x['training'][i, idx_max[start_trace : start_trace + block_size], :]).flatten()
            Xf[j,:] = fft(X)
        Xf = 2.0 / N_samples * np.abs(Xf[:, 0 : N_samples // 2])
        ax[i+2*N_vars].semilogy(freq, Xf.mean(axis=0), color=cmap[1], lw=1, label=f'H={H_values[1]:g} {inertia_units}')
       
    for a in ax:
        for side in 'right', 'top':
            a.spines[side].set_visible(False)
    for i,var_name in enumerate(var_names):
        ax[i].set_ylabel('Norm. ' + var_name)

    for i in range(N_vars):
        ax[i].set_ylim(ylim)
        ax[i+N_vars].set_ylim(ylim)
        ax[N_vars + i].plot([0, lim*1.05], [0,0], '--', lw=1, color=[.6,.6,.6])
        ax[2*N_vars + i].set_ylabel('Magnitude')
    ax[-1].set_xlabel('Frequency [Hz]')
    
    ax[N_vars-1].set_xlabel('Time [s]')
    ax[2*N_vars-1].set_xlabel('Fraction')
    ax[3*N_vars-1].legend(loc='upper right')
    fig.tight_layout()
    if gen_id is not None:
        fig.savefig(data_folder + f'/generator_{gen_id}.pdf');
    else:
        fig.savefig(data_folder + f'/bus_{bus_id}.pdf');

## Two area network

In [ ]:
H_values = [4,20]
for area in 1,2:
    data_folder = f'../data/two-area/var_H_area_{area}/two-area_D=2_DZA=60.0/'
    var_names_fmt = 'omegael_bus{}','Pe_bus{}'
    for bus in 7,9:
        var_names = tuple(var_name.format(bus) for var_name in var_names_fmt)
        plot_data(data_folder, H_values, var_names, bus_id=bus, area_id=area-1,
                  gen_areas_map=[['G1', 'G2'], ['G3', 'G4']],
                  gen_Pnom={f'G{i+1}': 1e9 for i in range(4)}, 
                  xlim_distr = 0.67, ylim=[-4.5,4.5])

## IEEE39

In [ ]:
bus_IDs = 3, 14, 17, 39
area = 2
H_G1 = 500
data_folder = f'../data/IEEE39/var_H_area_{area}/IEEE39_D=2_DZA=60.0/'
gen_areas_map = [['Pg31','Pg32','Pg39'], ['Pg33','Pg34','Pg35','Pg36'], ['Pg37','Pg38','Pg30']]
gen_Pnom = {gen: 100e6 for area in gen_areas_map for gen in area}
area_inertia = 'energy'
print(' ' * 58 + '{:^14s}'.format('INERTIA'))
print('=' * 77)
print('{:^56s} {:^6s} {:^6s} {:^6s}'.format('Filename', 'A1','A2','A3'))
print('=' * 77)
for data_file in glob.glob(data_folder + '/*_training_set.h5'):
    gen_IDs, gen_H, area_H = read_area_values(data_file, gen_areas_map, gen_Pnom, area_inertia)
    a,b = os.path.basename(data_file)[:30], os.path.basename(data_file)[-23:]
    print(f'{a}...{b} {area_H[0]:6.3f} {area_H[1]:6.3f} {area_H[2]:6.3f}')

In [ ]:
H_values = [8,14]
# var_names_fmt = 'omegael_bus{}','Pe_bus{}','Qe_bus{}','Vd_bus{}','Vq_bus{}'
var_names_fmt = 'Pe_bus{}','Vd_bus{}','Vq_bus{}'
for bus in bus_IDs:
    var_names = tuple(var_name.format(bus) for var_name in var_names_fmt)
    plot_data(data_folder, H_values, var_names, bus_id=bus, area_id=area-1,
              gen_areas_map=gen_areas_map, gen_Pnom=gen_Pnom, xlim_distr = 0.5, ylim=[-4,4])